# Overfitting(과적합)
"운영시스템에서도 동일한 성능을 보장할 수 있는가?"
---------------
샘플 데이터
* airquality 데이터
* mobile 데이터


## [1]단순한 모델 vs 복잡한 모델

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

airquality 데이터 셋을 이용하여 단순한 모델과 복잡한 모델을 만들어 봅시다!

In [ ]:
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/airquality_simple.csv'
air = pd.read_csv(path)

### 1)필요한 전처리 수행

In [ ]:
air.fillna(method = 'ffill', inplace=True)

In [ ]:
target = 'Ozone'
x = air.drop('Ozone', axis=1)
y = air.iloc[:, 0]
train_air_x, test_air_x, train_air_y, test_air_y = train_test_split(x, y, test_size=0.3
                                                                    ,shuffle = False, random_state=1)

### 2)가장 단순한 모델 : 평균 모델

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
train_air_x.shape

* 가장 단순한 모델(평균모델)
    * knn : k를 최대로 크게하면 평균 모델이 됨.

In [ ]:
# 모델링
n = 107
model = KNeighborsRegressor(n_neighbors = n) # train set의 행 수
model.fit(train_air_x, train_air_y)
result = model.predict(train_air_x)

# 결과 plotting
plt.plot(train_air_y)   # 실제값
plt.plot(result, color = 'r') # 예측값
plt.show()

### 3)(실습) 복잡도를 증가시켜가며 모델을 만들고 그림을 그려봅시다.

* 90, 70, 50, 30, 10, 5, 1
* k=가 1일때 가장 복잡한 모델!

In [ ]:
# 아래 90의 숫자를 줄여가며 모델링을 수행해 봅시다.
for n in [90, 70, 50, 30, 10, 5, 1] :
    model = KNeighborsRegressor(n_neighbors = n) # train set의 행 수
    model.fit(train_air_x, train_air_y)
    result = model.predict(train_air_x)

    # 결과 plotting
    plt.plot(train_air_y)   # 실제값
    plt.plot(result, color = 'r') # 예측값
    plt.title('k = {}'.format(n))
    plt.show()

## [2]복잡도와 과적합

### 00.환경 준비와 데이터 로딩

#### 01.라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#### 02.데이터 업로드 

In [ ]:
# mobile data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv"
mobile = pd.read_csv(path)

In [ ]:
# train set과 holdout(test set) 두 df로 나눕니다.
target = 'CHURN'
x = mobile.drop(target, axis=1)
y = mobile.loc[:, target]

### 20.데이터 준비

#### 21.변수 정리

In [ ]:
x.drop('id', axis = 1, inplace = True)

#### 22.NA 처리

In [ ]:
x.isna().sum()

#### 23.Feature Engineering

#### 24.Dummy Variable

In [ ]:
col_cat = ['REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL', 'CONSIDERING_CHANGE_OF_PLAN' ]

In [ ]:
# 범주형(명목형) 변수를 가변수로 만들고 붙여봅시다.

for v in col_cat :
    dummies = pd.get_dummies(x[v], prefix=v)
    x = pd.concat([x, dummies], axis=1)
    x.drop(v, axis = 1, inplace=True)

x.head()

#### 25.Scaling




#### 26.Data Split

In [ ]:
# train_val에서 train : val = 8 : 2
train_x, val_x, train_y, val_y = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
print(train_x.shape)
print(val_x.shape)

### Fitting Graph 그리기

* 모델의 복잡도를 증가시켜가면서
* train & validation set에 대해 예측 및 평가를 수행하고,
* 결과를 그래프로 그리기

#### 1) Decision Tree

* Decision Tree는 나무의 크기가 클 수록 복잡한 모델
* 크기를 결정하는 파라미터는 max_depth

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *

In [ ]:
result_train = [] # train set을 가지고 예측한 결과
result_val = [] # val set을 가지고 예측한 결과
depth = list(range(1,21))
for d in depth :
    model = DecisionTreeClassifier(max_depth = d)
    model.fit(train_x, train_y)
    train_pred, val_pred  = model.predict(train_x),model.predict(val_x)
    result_train.append(accuracy_score(train_y, train_pred))
    result_val.append(accuracy_score(val_y, val_pred))

In [ ]:
pd.DataFrame({'max_depth': depth,'train_acc':result_train, 'val_acc':result_val})

In [ ]:
plt.figure(figsize = (12,6))
plt.plot(depth, result_train, label = 'train_acc')
plt.plot(depth, result_val, label = 'val_acc')
plt.xlabel('Complexity(depth)')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

* 최적의 depth는?

### 실습 : KNN 알고리즘에 대해서 Fitting Graph를 그리시오.


* scaling : min-max 방식  
knn은 스케일링을 해줘야 합니다.

In [ ]:
col_x = list(x)

In [ ]:
# 필요한 함수 로딩
from sklearn.preprocessing import MinMaxScaler

# 함수 선언
scaler = MinMaxScaler()

# 함수 만들기
x2 = scaler.fit_transform(x)


In [ ]:
# 데이터프레임으로 다시 만듭시다.
x2 = pd.DataFrame(x2, columns = col_x)

**여기서부터 실습**  
* x2, y를 이용하여 train : val = 7:3으로 분할
* 위 코드를 참조하되, knn 알고리즘을 이용
* k값을 2~20까지 증가시키면서 train / val 정확도를 측정하고
* fitting graph를 그리시오

* 최적의 k 값은?